## This notebook launches the LatissCWFSAlign script, meant for the scriptQueue, but via a notebook.
##### This calculates focus offsets for the hexapod (and also decentering corrections)

In [1]:
import sys
import asyncio
import time
import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
%matplotlib inline
from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.observatory.control.utils import RotType

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [3]:
script = LatissCWFSAlign(index=1, remotes=True)  # this essentially calls the init method
# make sure all remotes etc are running
await script.start_task

DEBUG:Script.ATCS:atmcs: Adding all resources.
atmcs: Adding all resources.
DEBUG:Script.ATCS:atptg: Adding all resources.
atptg: Adding all resources.
DEBUG:Script.ATCS:ataos: Adding all resources.
ataos: Adding all resources.
DEBUG:Script.ATCS:atpneumatics: Adding all resources.
atpneumatics: Adding all resources.
DEBUG:Script.ATCS:athexapod: Adding all resources.
athexapod: Adding all resources.
DEBUG:Script.ATCS:atdome: Adding all resources.
atdome: Adding all resources.
DEBUG:Script.ATCS:atdometrajectory: Adding all resources.
atdometrajectory: Adding all resources.
DEBUG:Script.LATISS:atcamera: Adding all resources.
atcamera: Adding all resources.
DEBUG:Script.LATISS:atspectrograph: Adding all resources.
atspectrograph: Adding all resources.
DEBUG:Script.LATISS:atheaderservice: Adding all resources.
atheaderservice: Adding all resources.
DEBUG:Script.LATISS:atarchiver: Adding all resources.
atarchiver: Adding all resources.
INFO:Script:latiss_cwfs_align initialized!
latiss_cwfs_a

In [ ]:
# set wrap strategy
# this is required until the ATPtg is updated to not configure the mount for maximum time on target
#script.atcs.rem.atptg.cmd_raDecTarget.set(azWrapStrategy=1)  # 1 does not unwrap, 0 unwraps

In [ ]:
# ATAOS must be on and corrections enabled, do as follows if required
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# Slew to an object if not already there.
#await script.atcs.slew_object('HD 94340', rot_type=RotType.Parallactic)

In [ ]:
await latiss.take_object(5.0, 1)

## Emulate how the scriptQueue launches scripts
##### Start here if re-running the script after a correction

In [13]:
configuration = yaml.safe_dump({"filter": 'RG610', 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook'})
print(configuration)

dataPath: /project/shared/auxTel/rerun/quickLook
exposure_time: 20
filter: RG610
grating: empty_1



In [ ]:
# May want to do an XY offset
# await script.atcs.offset_xy(x=0, y=-60)

In [14]:
# Needed if no slew
script.atcs.dome_az_in_position.set()

In [15]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

INFO:Script:Using binning factor of 1
Using binning factor of 1


In [16]:
# This will run the cwfs multiple times until it converges,
# then apply the calculated offsets
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10
results = await script.arun()

DEBUG:Script:CWFS iteration 1 starting...
CWFS iteration 1 starting...
Intra/Extra images not taken. Running take image sequence.
DEBUG:Script:Moving to intra-focal position
Moving to intra-focal position
DEBUG:Script.LATISS:imagetype: ENGTEST, skip TCS synchronization.
imagetype: ENGTEST, skip TCS synchronization.
DEBUG:Script.LATISS:ENGTEST 0001 - 0001
ENGTEST 0001 - 0001
DEBUG:Script:Moving to extra-focal position
Moving to extra-focal position
DEBUG:Script:Taking extra-focal image
Taking extra-focal image
DEBUG:Script.LATISS:imagetype: ENGTEST, skip TCS synchronization.
imagetype: ENGTEST, skip TCS synchronization.
DEBUG:Script.LATISS:ENGTEST 0001 - 0001
ENGTEST 0001 - 0001
INFO:Script:intraImage expId for target: 2021052500210
intraImage expId for target: 2021052500210
INFO:Script:extraImage expId for target: 2021052500211
extraImage expId for target: 2021052500211
INFO:Script:angle used in cwfs algorithm is -117.20191319872319
angle used in cwfs algorithm is -117.20191319872319
D

In [17]:
focus_offset_summary = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()

In [18]:
print(focus_offset_summary)

private_revCode: d8296953, private_sndStamp: 1621995516.6562276, private_rcvStamp: 1621995516.6567218, private_seqNum: 301, private_identity: ATAOS, private_origin: 153650, private_host: 0, total: -0.7798266410827637, userApplied: 0.013373331166803837, filter: -0.01600000075995922, disperser: 0.0, wavelength: 0.0, priority: 0
logMessage DDS read queue is filling: 51 of 100 elements
logMessage DDS read queue is filling: 28 of 100 elements
logMessage DDS read queue is filling: 55 of 100 elements


In [ ]:
# This will just run the CWFS one time
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10
results = await script.run_cwfs()

In [ ]:
# show donuts and centroids
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title(f"intra visitID - {script.intra_visit_id}")
ax11.imshow(script.I1[0].image0)
ax11.contour(script.algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title(f"extra visitID - {script.extra_visit_id}")
ax12.imshow(script.I2[0].image0)
ax12.contour(script.algo.pMask) 

In [ ]:
# Apply calculated focus offset
calculated_hexapod_focus_offset = results['hex_offset'][2]
print(f'Applying focus offset of {calculated_hexapod_focus_offset}')
await script.atcs.rem.ataos.cmd_offset.set_start(z=calculated_hexapod_focus_offset)

In [ ]:
await script.atcs.slew_object('HD185975', rot_type=2)

In [ ]:
await script.latiss.take_object(5.0)

# Stop here unless a re-reduction of the doughnuts is required.

#### If you want to re-reduce data then use the below cells

In [ ]:
# Show which files/parameters were taken in the sequence above
print(f'intra_visit_id is {script.intra_visit_id}')
print(f'extra_visit_id is {script.extra_visit_id}')
print(f'angle is {script.angle}')

In [ ]:
# If desired then different filenames can be manually input here
#script.intra_visit_id=2021011900169 
#script.extra_visit_id=2021011900170 
#script.angle=-91.56748047249727

In [ ]:
# reruns reduction part only. ALL 3 fields above must be set! 
rerun_results = await script.run_cwfs()